# Roughing in Architecture
#### Employ Training Data for Outlining 

In [83]:
import pandas as pd
import requests
import re
from openai import OpenAI
import instructor
from instructor import patch
from pydantic import BaseModel
from typing import Dict

class WikipediaAPICall(BaseModel):
    action: str
    format: str
    prop: str
    titles: str
    choices: str

client = patch(OpenAI())

df = pd.read_csv('train.csv')
row = df.iloc[0]

test_item = row['prompt']  + ' ' + row['A'] + ' ' + row['B'] + ' ' + row['C'] + ' ' + row['D'] + ' ' + row['E'] + ' ' + row['answer']

wiki_api_call = WikipediaAPICall(
    action="query",
    format="json",
    prop="extracts",
    titles=test_item,
    choices="content"
)

client = instructor.patch(client)

class SearchTerms(BaseModel):
    first: str
    second: str
    third: str
    
resp = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
            "role": "user", 
            "content": f"Read the MCQ and give the top three terms you would use to search for an answer: {test_item}",
        },
    ],
    response_model=SearchTerms
)
resp
print(resp)

first='Modified Newtonian Dynamics' second='observed missing baryonic mass' third='galaxy clusters'


In [64]:
import re
def clean_string(s):
  # Remove everything inside parentheses
  s = re.sub(r'\([^()]*\)', '', s)
  
  # Remove all non-alphabetical characters except spaces
  s = re.sub(r'[^a-zA-Z ]', '', s)
  
  # Remove spaces from the end of the string
  s = s.rstrip()
  
  return s
search_term = clean_string(resp.first)
print(search_term)


Modified Newtonian Dynamics


In [77]:
import requests

def print_first_two_wikipedia_pages_content(search_query):
    S = requests.Session()

    URL = "https://en.wikipedia.org/w/api.php"

    SEARCHQUERY = search_query

    PARAMS = {
        "action": "opensearch",
        "format": "json",
        "search": SEARCHQUERY,
        "limit": 2,
    }

    response = S.get(url=URL, params=PARAMS)
    data = response.json()

    page_titles = data[1]

    for title in page_titles:
        PARAMS = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        }

        response = S.get(url=URL, params=PARAMS)
        data = response.json()

        pages = data["query"]["pages"]

        for k, v in pages.items():
            print(v["extract"])

print_first_two_wikipedia_pages_content(search_term)

Modified Newtonian dynamics (MOND) is a hypothesis that proposes a modification of Newton's second law to account for observed properties of galaxies. It is an alternative to the hypothesis of dark matter in terms of explaining why galaxies do not appear to obey the currently understood laws of physics.
Created in 1982 and first published in 1983 by Israeli physicist Mordehai Milgrom, the hypothesis' original motivation was to explain why the velocities of stars in galaxies were observed to be larger than expected based on Newtonian mechanics. Milgrom noted that this discrepancy could be resolved if the gravitational force experienced by a star in the outer regions of a galaxy was proportional to the square of its centripetal acceleration (as opposed to the centripetal acceleration itself, as in Newton's second law) or alternatively, if gravitational force came to vary inversely linearly with radius (as opposed to the inverse square of the radius, as in Newton's law of gravity). MOND d

In [81]:
def print_first_two_wikipedia_pages_content(search_query):
    S = requests.Session()

    URL = "https://en.wikipedia.org/w/api.php"

    SEARCHQUERY = search_query

    PARAMS = {
        "action": "opensearch",
        "format": "json",
        "search": SEARCHQUERY,
        "limit": 2,
    }

    response = S.get(url=URL, params=PARAMS)
    data = response.json()

    page_titles = data[1]

    content = ""
    for title in page_titles:
        PARAMS = {
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        }

        response = S.get(url=URL, params=PARAMS)
        data = response.json()

        pages = data["query"]["pages"]

        for k, v in pages.items():
            content += v["extract"]
    
    return content
def find_terms_in_same_paragraph(text, term1, term2):
  paragraphs = re.split(r'\s*\n\s*', text)
  matching_paragraphs = []
  for paragraph in paragraphs:
      if re.search(fr'\b{term1}\b', paragraph, re.IGNORECASE) and re.search(fr'\b{term2}\b', paragraph, re.IGNORECASE):
          matching_paragraphs.append(paragraph)
  if not matching_paragraphs:
      return "No paragraph found containing the specified terms."
  return matching_paragraphs



text = print_first_two_wikipedia_pages_content(search_term)

paragraphs = find_terms_in_same_paragraph(text, resp.second, resp.third)
for i, paragraph in enumerate(paragraphs):
    print(f"Paragraph {i+1}:\n{paragraph}\n")


Paragraph 1:
MOND is an example of a class of theories known as modified gravity, and is an alternative to the hypothesis that the dynamics of galaxies are determined by massive, invisible dark matter halos. Since Milgrom's original proposal, proponents of MOND have claimed to successfully predict a variety of galactic phenomena that they state are difficult to understand as consequences of dark matter.Though MOND explains the anomalously great rotational velocities of galaxies at their perimeters, it does not fully explain the velocity dispersions of individual galaxies within galaxy clusters. MOND reduces the discrepancy between the velocity dispersions and clusters' observed missing baryonic mass from a factor of around 10 to a factor of about 2. However, the residual discrepancy cannot be accounted for by MOND, requiring that other explanations close the gap such as the presence of as-yet undetected missing baryonic matter.The accurate measurement of the speed of gravitational wave